In [ ]:
# pytorch 模型 保存完整模型，triton推理

In [2]:
! apt update -y
! apt install -y wget unzip
! pip install torchvision torch

Hit:1 http://mirrors.tencent.com/ubuntu bionic InRelease
Hit:2 http://mirrors.tencent.com/ubuntu bionic-security InRelease              
Hit:3 http://mirrors.tencent.com/ubuntu bionic-updates InRelease               
Hit:4 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease      
Hit:5 https://deb.nodesource.com/node_16.x bionic InRelease                 
Reading package lists... Done                
Building dependency tree       
Reading state information... Done
102 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
wget is already the newest version (1.19.4-1ubuntu2.2).
Suggested packages:
  zip
The following NEW packages will be installed:
  unzip
0 upgraded, 1 newly installed, 0 to remove and 102 not upgraded.
Need to get 168 kB of archives.
After this operation, 567 kB of additional disk space will be used.
Get:1 http://mirrors.tencent.com/ubuntu bionic-sec

In [3]:
# triton 直接推理 torch模型
! rm -rf resnet50.pth && wget https://docker-76009.sz.gfp.tencent-cloud.com/github/cube-studio/inference/resnet50.pth

--2022-07-30 12:23:36--  https://docker-76009.sz.gfp.tencent-cloud.com/github/cube-studio/inference/resnet50.pth
Resolving docker-76009.sz.gfp.tencent-cloud.com (docker-76009.sz.gfp.tencent-cloud.com)... 43.137.222.31, 175.27.38.240, 43.137.221.26, ...
Connecting to docker-76009.sz.gfp.tencent-cloud.com (docker-76009.sz.gfp.tencent-cloud.com)|43.137.222.31|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 102530333 (98M) [application/octet-stream]
Saving to: ‘resnet50.pth’

resnet50.pth        100%[===================>]  97.78M  6.42MB/s    in 15s     

2022-07-30 12:23:52 (6.54 MB/s) - ‘resnet50.pth’ saved [102530333/102530333]



In [4]:
from torchvision import models
# from torchsummary import summary
import torch
from torchvision.models.resnet import resnet50
device = torch.device('cpu')

model = resnet50(pretrained=False, progress=True)   # 创建模型
model.load_state_dict(torch.load("resnet50.pth",map_location='cpu')) # 加载模型参数

# 查看模型结构
print(model)

# 查看模型参数
for name, parameters in model.named_parameters():
    print(name, ':', parameters.size())

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [5]:
!rm -rf resnet50.onnx torchscript.pt

model.eval()     # 设置模型为推理模式

# 导出网络到ONNX，需提供输入输出名
dummy_input = torch.randn(1, 3, 224, 224).to(device)  # 输入样本
torch.onnx.export(
    model,dummy_input,"resnet50.onnx",
    opset_version=13,           # 转为onnx的版本
    do_constant_folding=True,   # 是否执行常量折叠优化
    input_names=["input_name"],     # 输入名
    output_names=["output_name"],   # 输出名
    # dynamic_axes={
    #     "input":{0:"batch_size"},   # 批处理变量
    #     "output":{0:"batch_size"}
    # },
    # dynamic_axes={'input_name': [2, 3], 'output_name': [2, 3]}   # 动态size的输入输出的维度
)


# 保存TORCHSCRIPT
# torch script保存的模型，目前不提供输入和输出的端口的命名，
# 因此在配置文件中，输入和输出端口的名字必须按照如下命名： "INPUT__0", "INPUT__1" and "OUTPUT__0", "OUTPUT__1"
dummy_input = torch.randn(1, 3, 224, 224).to(device)  # 输入样本
traced_cell = torch.jit.trace(model, dummy_input)
traced_cell.save("resnet50-torchscript.pt")


In [7]:
# 配置pytorch模型 config.pbtxt

In [ ]:
name: "resnet50"
platform: "pytorch_libtorch"
max_batch_size : 0
input [
  {
    name: "input"
    data_type: TYPE_FP32
    format: FORMAT_NCHW
    dims: [ 3, 224, 224 ]
    reshape { shape: [ 1, 3, 224, 224 ] }
  }
]
output [
  {
    name: "output"
    data_type: TYPE_FP32
    dims: [ 1000 ]
    reshape { shape: [ 1, 1000, 1, 1 ] }
  }
]

In [ ]:
# pytorch 转为onnx 使用triton 推理

In [8]:
! pip install onnx netron
# 查看网络模型结构
import netron
modelPath = "resnet50.onnx"
netron.start(modelPath)

Looking in indexes: https://mirrors.tencent.com/pypi/simple/, https://mirrors.tencent.com/repository/pypi/tencent_pypi/simple
     |████████████████████████████████| 1.5 MB 811 kB/s            
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.
Serving 'resnet50.onnx' at http://localhost:8080


('localhost', 8080)

In [ ]:
# 配置onnx模型 config.pbtxt

In [ ]:
name: "resnet50"
platform: "onnxruntime_onnx"
backend: "onnxruntime"
max_batch_size : 0

input [
  {
    name: "data"
    data_type: TYPE_FP32
    format: FORMAT_NCHW
    dims: [ 3, 224, 224 ]
    reshape { shape: [ 1, 3, 224, 224 ] }
  }
]
output [
  {
    name: "resnetv17_dense0_fwd"
    data_type: TYPE_FP32
    dims: [ 1000 ]
    reshape { shape: [ 1, 1000 ] }
  }
]

parameters { key: "intra_op_thread_count" value: { string_value: "30" } }
parameters { key: "execution_mode" value: { string_value: "1" } }
parameters { key: "inter_op_thread_count" value: { string_value: "30" } }



In [ ]:
# 客户端请求
import datetime
import argparse
from functools import partial
import os
import sys

from PIL import Image
import numpy as np
from attrdict import AttrDict
import struct
import tritonclient.grpc as grpcclient
import tritonclient.grpc.model_config_pb2 as mc
import tritonclient.http as httpclient
from tritonclient.utils import InferenceServerException
from tritonclient.utils import triton_to_np_dtype
from urllib.parse import quote, quote_plus
import queue
import json,time
import pysnooper
# pip3 install Pillow
import requests

# @pysnooper.snoop()
def parse_model(model_metadata, model_config):
    """
    Check the configuration of a model to make sure it meets the
    requirements for an image classification network (as expected by
    this client)
    """
    if len(model_metadata.inputs) != 1:
        raise Exception("expecting 1 input, got {}".format(
            len(model_metadata.inputs)))
    if len(model_metadata.outputs) != 1:
        raise Exception("expecting 1 output, got {}".format(
            len(model_metadata.outputs)))

    if len(model_config.input) != 1:
        raise Exception(
            "expecting 1 input in model configuration, got {}".format(
                len(model_config.input)))

    input_metadata = model_metadata.inputs[0]
    input_config = model_config.input[0]
    output_metadata = model_metadata.outputs[0]

    if output_metadata.datatype != "FP32":
        raise Exception("expecting output datatype to be FP32, model '" +
                        model_metadata.name + "' output type is " +
                        output_metadata.datatype)

    # Output is expected to be a vector. But allow any number of
    # dimensions as long as all but 1 is size 1 (e.g. { 10 }, { 1, 10
    # }, { 10, 1, 1 } are all ok). Ignore the batch dimension if there
    # is one.
    output_batch_dim = (model_config.max_batch_size > 0)
    non_one_cnt = 0
    for dim in output_metadata.shape:
        if output_batch_dim:
            output_batch_dim = False
        elif dim > 1:
            non_one_cnt += 1
            if non_one_cnt > 1:
                raise Exception("expecting model output to be a vector")

    # Model input must have 3 dims, either CHW or HWC (not counting
    # the batch dimension), either CHW or HWC
    input_batch_dim = (model_config.max_batch_size > 0)
    expected_input_dims = 3 + (1 if input_batch_dim else 0)
    if len(input_metadata.shape) != expected_input_dims:
        raise Exception(
            "expecting input to have {} dimensions, model '{}' input has {}".
                format(expected_input_dims, model_metadata.name,
                       len(input_metadata.shape)))

    if type(input_config.format) == str:
        FORMAT_ENUM_TO_INT = dict(mc.ModelInput.Format.items())
        input_config.format = FORMAT_ENUM_TO_INT[input_config.format]

    if ((input_config.format != mc.ModelInput.FORMAT_NCHW) and
            (input_config.format != mc.ModelInput.FORMAT_NHWC)):
        raise Exception("unexpected input format " +
                        mc.ModelInput.Format.Name(input_config.format) +
                        ", expecting " +
                        mc.ModelInput.Format.Name(mc.ModelInput.FORMAT_NCHW) +
                        " or " +
                        mc.ModelInput.Format.Name(mc.ModelInput.FORMAT_NHWC))

    if input_config.format == mc.ModelInput.FORMAT_NHWC:
        h = input_metadata.shape[1 if input_batch_dim else 0]
        w = input_metadata.shape[2 if input_batch_dim else 1]
        c = input_metadata.shape[3 if input_batch_dim else 2]
    else:
        c = input_metadata.shape[1 if input_batch_dim else 0]
        h = input_metadata.shape[2 if input_batch_dim else 1]
        w = input_metadata.shape[3 if input_batch_dim else 2]

    return (model_config.max_batch_size, input_metadata.name,
            output_metadata.name, c, h, w, input_config.format,
            input_metadata.datatype)


def preprocess(img, format, dtype, c, h, w, scaling, protocol):
    """
    Pre-process an image to meet the size, type and format
    requirements specified by the parameters.
    """
    # np.set_printoptions(threshold='nan')

    if c == 1:
        sample_img = img.convert('L')
    else:
        sample_img = img.convert('RGB')

    resized_img = sample_img.resize((w, h), Image.BILINEAR)
    resized = np.array(resized_img)
    if resized.ndim == 2:
        resized = resized[:, :, np.newaxis]

    npdtype = triton_to_np_dtype(dtype)
    typed = resized.astype(npdtype)

    if scaling == 'INCEPTION':
        scaled = (typed / 127.5) - 1
    elif scaling == 'VGG':
        if c == 1:
            scaled = typed - np.asarray((128,), dtype=npdtype)
        else:
            scaled = typed - np.asarray((123, 117, 104), dtype=npdtype)
    else:
        scaled = typed

    # Swap to CHW if necessary
    if format == mc.ModelInput.FORMAT_NCHW:
        ordered = np.transpose(scaled, (2, 0, 1))
    else:
        ordered = scaled

    # Channels are in RGB order. Currently model configuration data
    # doesn't provide any information as to other channel orderings
    # (like BGR) so we just assume RGB.
    return ordered


def convert_http_metadata_config(_metadata, _config):
    _model_metadata = AttrDict(_metadata)
    _model_config = AttrDict(_config)

    return _model_metadata, _model_config




model_name='resnet50'
model_version='1'
url = 'resnet50-tensorrt.service.kfserving.woa.com'
classes=1000
if __name__ == '__main__':

    triton_client = httpclient.InferenceServerClient(url=url, verbose=False, concurrency=1)

    # 获取模型的配置信息
    model_metadata = triton_client.get_model_metadata(model_name=model_name, model_version=model_version)
    # print(model_metadata)
    model_config = triton_client.get_model_config(model_name=model_name, model_version=model_version)
    # print(model_config)
    model_metadata, model_config = convert_http_metadata_config(model_metadata, model_config)
    print(model_metadata,model_config)
    max_batch_size, input_name, output_name, c, h, w, format, dtype = parse_model(model_metadata, model_config)
    # max_batch_size, input_name, output_name, c, h, w, format, dtype = 0,'data','resnetv17_dense0_fwd',3,224,224,2,'FP32'
    print(max_batch_size, input_name, output_name, c, h, w, format, dtype)

    filename = 'smallcat.jpg'

    image_data = preprocess(Image.open(filename), format, dtype, c, h, w, None,'http')
    batched_image_data = image_data

    # Send request
    input = httpclient.InferInput(input_name, batched_image_data.shape, dtype)
    input.set_data_from_numpy(batched_image_data)
    output = httpclient.InferRequestedOutput(output_name, class_count=classes)


    response = triton_client.infer(model_name,[input],request_id=str(time.time()),model_version=model_version,outputs=[output])
    output_array = response.as_numpy(output_name)
    print(output_array)

